# Homework 19: Blockchain with Python

## Imports

In [1]:
# Import dependencies
import subprocess
import json
import os
import pprint
from dotenv import load_dotenv

# Load and set environment variables
load_dotenv('token.env')
mnemonic=os.getenv("mnemonic")

# Import constants.py and necessary functions from bit and web3
from constants import *
from bit import wif_to_key
from bit import PrivateKeyTestnet

from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account

### Check constants and mnemonic

In [2]:
# Check imported constants and mnemonic
print(ETH, BTCTEST, BTC)
print(mnemonic)

eth btc-test btc
alter puppy choice hello almost trophy report innocent letter canyon actress now


## Derive Wallet Keys

### Tests

In [3]:
# Test `./derive` script from README
command = 'php ./derive --key=xprv9zbB6Xchu2zRkf6jSEnH9vuy7tpBuq2njDRr9efSGBXSYr1QtN8QHRur28QLQvKRqFThCxopdS1UD61a5q6jGyuJPGLDV9XfYHQto72DAE8 --cols=path,address --coin=ZEC --numderive=3 -g --format=json'

p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
output, err = p.communicate()
p_status = p.wait()

keys = json.loads(output)
keys

[{'path': 'm/0', 'address': 't1V1Qp41kbHn159hvVXZL5M1MmVDRe6EdpA'},
 {'path': 'm/1', 'address': 't1Tw6iqFY1g9dKeAqPDAncaUjha8cn9SZqX'},
 {'path': 'm/2', 'address': 't1VGTPzBSSYd27GF8p9rGKGdFuWekKRhug4'}]

#### BTC-test derive_wallet function

In [4]:
# test = "BTC-test"

In [5]:
# # Create a function called `derive_wallets`
# # Derive 'btc-test' wallet keys
# # '--coin' flag does not accept the BTCTEST constant when passed in the function by itself
# def derive_btc(command, stdout=subprocess.PIPE, shell=True):
#     command = 'php ./derive -g --mnemonic=mnemonic --coin="btc-test" --numderive=3 --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'
#     p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     output, err = p.communicate()
#     p_status = p.wait()
#     return json.loads(output)

# derive_btc(command)

#### ETH derive_wallet function

In [6]:
# # Derive 'eth' wallet keys
# def derive_eth(command, stdout=subprocess.PIPE, shell=True):
#     command = 'php ./derive -g --mnemonic=mnemonic --coin=ETH --numderive=3 --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'
#     p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     output, err = p.communicate()
#     p_status = p.wait()
#     return json.loads(output)

# derive_eth(command)

### `derive_wallets` function

In [9]:
# Derive both sets of wallet keys in one function
def derive_wallets(coin, stdout=subprocess.PIPE, shell=True):
    command = f'php ./derive -g --mnemonic=mnemonic --coin={coin} --numderive=3 --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

### `coins` dict

In [10]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(ETH),
    BTCTEST: derive_wallets(BTCTEST),
    }

coins

{'eth': [{'address': '0x3De0A2fD4A90f9A160ebb2B8711192D1F0eB339D',
   'index': 0,
   'path': "m/44'/60'/0'/0/0",
   'privkey': '0x82d63adc8be924a92ef77c6e3286c4e80eaebad63ba2b91c27801a62ba2f9410',
   'pubkey': '02c93d99b9686cabed081f1eb262b81a435ae2816ba940e0fea550a36603d8077b',
   'pubkeyhash': 'a90e4da9a141168d3e0ac3821015190a79df729c',
   'xprv': 'xprvA3W3uzhjas8mpZrqS7oW9ar2SWxLGhwcPFT7S2j7VM8e9PbaYcfSJo4aDdsjd7yi9iPcKkvxg7TzmeNJniz84Y4rYgTaarwiMk66gZNwj3P',
   'xpub': 'xpub6GVQKWEdREh533wJY9LWWinkzYnpgAfTkUNiER8j3gfd2Bvj69ygrbP44uSWSEu9BwzQ2mboHQp17puXxT3y9xnXCt256vtjkzV2caX1xeT'},
  {'address': '0xc5557310a53b90f18E926431B1896c29Df0dcc1a',
   'index': 1,
   'path': "m/44'/60'/0'/0/1",
   'privkey': '0xf0c147a367c5732f750e0dc072540f71ec3e3a87fafee9e7516982861e9e259e',
   'pubkey': '0299b67384c634d60ac57d9efc57a93b3a2d87b9eb63e3025652de882c269f2480',
   'pubkeyhash': '8703eefc2cdcf91b844d7ec8090d2e89d092de58',
   'xprv': 'xprvA3W3uzhjas8msDSBJH94x7mqj53hfhCweGzWNRqsHamAi3MrZh4cDT4V

In [11]:
# Check access to `coins` dict
print(f"ETH private key: {coins[ETH][0]['privkey']}")
print(f"BTCTEST private key: {coins[BTCTEST][0]['privkey']}")

ETH private key: 0x82d63adc8be924a92ef77c6e3286c4e80eaebad63ba2b91c27801a62ba2f9410
BTCTEST private key: cVAgDoahzo5Ucdv8ioxxubvmE6LfU3ZQgJ21UvY4eC1xi6WkvnXe


## Linking Transaction Signing Libraries

### `priv_key_to_account` function

In [12]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin):
    if coin == ETH:
        return Account.privateKeyToAccount(coins[coin][0]['privkey'])
    if coin == BTCTEST:
        return PrivateKeyTestnet(coins[coin][0]['privkey'])

#### Tests

In [13]:
# # Tests for syntax and object types
# eprivkey = Account.privateKeyToAccount(coins[ETH][0]['privkey'])
# print(f'{eprivkey} || {type(eprivkey)}\n')

# bprivkey = coins[BTCTEST][0]['privkey']
# bwifkey = wif_to_key(bprivkey)
# print(f'{bwifkey} || {type(bwifkey)}')
# print(f'{PrivateKeyTestnet(bprivkey)} || {type(bwifkey)}\n')

# print(priv_key_to_account(ETH, 0))
# print(priv_key_to_account(BTCTEST, 0))

In [14]:
# # Test bit functions
# key = priv_key_to_account(BTCTEST, 0)

# print(f'# BTCTEST Balance: {key.get_balance("btc")}\n')

# print(f'# BTCTEST Balance in USD: ${key.balance_as("usd")}\n')

# print(f'# BTCTEST Transactions:\n{key.get_transactions()[:3]}\n')

# print(f'# BTCTEST Unspent:\n{key.get_unspents()}\n')

# # PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [15]:
# # Test web3 functions
# w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
# w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# def create_raw_tx(account, recipient, amount):
#     gasEstimate = w3.eth.estimateGas(
#         {"from": account.address, "to": recipient, "value": amount}
#     )
#     return {
#         "from": account.address,
#         "to": recipient,
#         "value": amount,
#         "gasPrice": w3.eth.gasPrice,
#         "gas": gasEstimate,
#         "nonce": w3.eth.getTransactionCount(account.address),
#     }


### `create_tx` function

In [16]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        # Convert ETH to WEI
        value = w3.toWei(amount, coin)
        gasEstimate = w3.eth.estimateGas({'from': account, 'to': to, 'amount': value})
        return {
            'from': account,
            'to': to,
            'value': value,
            'gasPrice': w3.eth.gasPrice,
            'gas': gasEstimate,
            'nonce': w3.eth.getTransactionCount(account),
            'chainID': w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

### `send_tx` function

In [17]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        tx = create_tx(coin, account.address, to, amount)
        signed_tx = account.signTransaction(tx)
        return w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    if coin == BTCTEST:
        tx = create_tx(coin, account.address, to, amount)
        signed_tx = account.sign_transaction(tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

---

# *Work in Progress*

## Send some transactions